In [60]:
import requests
import pandas as pd
import yfinance as yf
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


## (1) Web Scrapping ibovespa website

In [61]:
# In order to get the last chromedriver version we use .install method
# Inicialize ChromeDriver 
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install()))
url = 'https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm'
driver.get(url)

In [62]:
# Find "id = bvmf_iframe"
driver.find_element(By.ID, "bvmf_iframe")

<selenium.webdriver.remote.webelement.WebElement (session="4c91fd53d997e916495302416a948eb9", element="f.EB99BEA10C9AD7B4D548F69F52070FAA.d.7787E4D4F3EA70C513FDFC28DB85DF37.e.28")>

In [ ]:
# Wait the page to finishy loading the content
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "bvmf_iframe"))
)

# Change the context
driver.switch_to.frame(iframe)

In [64]:
# Wait for the overlay to disappear before clicking the download button
WebDriverWait(driver, 10).until(
    EC.invisibility_of_element_located((By.CLASS_NAME, "ot-sdk-row"))
)

# Finding the buttom "download"
download = driver.find_element(By.XPATH, "/html/body/app-root/app-day-portfolio/div/div/div[1]/form/div[2]/div/div[2]/div/div/div[1]/div[2]/p/a")


In [65]:
#
driver.execute_script("arguments[0].scrollIntoView(true);", download)
download.click()

## (2) Colect and transform stocks data

In [83]:
# Load ibov excel file
# Fix file: transform into xlsx with python
df_ibov = pd.read_excel("C:/Users/yohan/Downloads/IBOVDia_02-12-24.xlsx", skiprows=[0,88,89])
display(df_ibov.head())
display(df_ibov.tail())

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,ALOS3,ALLOS,ON NM,502481592,0.479
1,ALPA4,ALPARGATAS,PN N1,166460180,0.051
2,ABEV3,AMBEV S/A,ON,4394835131,2.710
3,ASAI3,ASSAI,ON NM,1349687675,0.431
4,AURE3,AUREN,ON NM,291727616,0.139


,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
81,VAMO3,VAMOS,ON EJ NM,485166826,0.131
82,VBBR3,VIBRA,ON NM,1023392529,1.022
83,VIVA3,VIVARA S.A.,ON NM,125823025,0.140
84,WEGE3,WEG,ON NM,1269032828,3.315
85,YDUQ3,YDUQS PART,ON NM,290010465,0.130


In [108]:
# Tickers list
tickers = list(df_ibov["Código"].unique())

# Adding ".SA" at the end
for i in range(len(tickers)):
    tickers[i] = tickers[i][:] + ".SA"

# View
print(tickers)

# Get historical data
raw_data = yf.download(tickers, start="2014-01-01", end="2024-01-01")

# View
display(raw_data.head())
display(raw_data.tail())


['ALOS3.SA', 'ALPA4.SA', 'ABEV3.SA', 'ASAI3.SA', 'AURE3.SA', 'AZUL4.SA', 'AZZA3.SA', 'B3SA3.SA', 'BBSE3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BRAP4.SA', 'BBAS3.SA', 'BRKM5.SA', 'BRAV3.SA', 'BRFS3.SA', 'BPAC11.SA', 'CXSE3.SA', 'CRFB3.SA', 'CCRO3.SA', 'CMIG4.SA', 'COGN3.SA', 'CPLE6.SA', 'CSAN3.SA', 'CPFE3.SA', 'CMIN3.SA', 'CVCB3.SA', 'CYRE3.SA', 'ELET3.SA', 'ELET6.SA', 'EMBR3.SA', 'ENGI11.SA', 'ENEV3.SA', 'EGIE3.SA', 'EQTL3.SA', 'EZTC3.SA', 'FLRY3.SA', 'GGBR4.SA', 'GOAU4.SA', 'NTCO3.SA', 'HAPV3.SA', 'HYPE3.SA', 'IGTI11.SA', 'IRBR3.SA', 'ISAE4.SA', 'ITSA4.SA', 'ITUB4.SA', 'JBSS3.SA', 'KLBN11.SA', 'RENT3.SA', 'LREN3.SA', 'LWSA3.SA', 'MGLU3.SA', 'MRFG3.SA', 'BEEF3.SA', 'MRVE3.SA', 'MULT3.SA', 'PCAR3.SA', 'PETR3.SA', 'PETR4.SA', 'RECV3.SA', 'PRIO3.SA', 'PETZ3.SA', 'RADL3.SA', 'RAIZ4.SA', 'RDOR3.SA', 'RAIL3.SA', 'SBSP3.SA', 'SANB11.SA', 'STBP3.SA', 'SMTO3.SA', 'CSNA3.SA', 'SLCE3.SA', 'SUZB3.SA', 'TAEE11.SA', 'VIVT3.SA', 'TIMS3.SA', 'TOTS3.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'VAMO3.SA', 'VBBR3.SA

[*********************100%%**********************]  86 of 86 completed

2 Failed downloads:
['BRAV3.SA', 'ISAE4.SA']: YFChartError("%ticker%: Data doesn't exist for startDate = 1388541600, endDate = 1704078000")


Price       Adj Close                                                \
Ticker       ABEV3.SA ALOS3.SA  ALPA4.SA ASAI3.SA AURE3.SA AZUL4.SA   
Date                                                                  
2014-01-02  11.381043      NaN  8.090009      NaN      NaN      NaN   
2014-01-03  11.138892      NaN  8.003328      NaN      NaN      NaN   
2014-01-06  10.994913      NaN  7.887759      NaN      NaN      NaN   
2014-01-07  11.341774      NaN  7.650835      NaN      NaN      NaN   
2014-01-08  11.453032      NaN  7.662395      NaN      NaN      NaN   

Price                                                ...   Volume            \
Ticker       AZZA3.SA  B3SA3.SA  BBAS3.SA  BBDC3.SA  ... TOTS3.SA  UGPA3.SA   
Date                                                 ...                      
2014-01-02  23.257196  2.338520  6.003547  7.508717  ...  1588799   4658305   
2014-01-03  23.943472  2.360624  6.066081  7.340267  ...   545067  11029561   
2014-01-06  23.585083  2.300945  5.976029  7.390010  ...  1110020   6514508   
2014-01-07  22.624296  2.278842  5.873466  7.250257  ...   798768   5774371   
2014-01-08  21.953274  2.331889  5.755898  7.228943  ...  1775610   4363118   

Price                                                                       \
Ticker      USIM5.SA VALE3.SA VAMO3.SA VBBR3.SA VIVA3.SA VIVT3.SA WEGE3.SA   
Date                                                                         
2014-01-02   7429100  4395300      NaN      NaN      NaN    26200  1705548   
2014-01-03  18385800  9207100      NaN      NaN      NaN    38700  4189848   
2014-01-06   5677900  4527200      NaN      NaN      NaN     6200  2741180   
2014-01-07   5058700  4352500      NaN      NaN      NaN    26600  3304964   
2014-01-08   4664100  2697700      NaN      NaN      NaN    32600  4343976   

Price                
Ticker     YDUQ3.SA  
Date                 
2014-01-02  2474500  
2014-01-03  8302300  
2014-01-06  2633000  
2014-01-07  1673000  
2014-01-08  1978000  

[5 rows x 516 columns]

Price      Adj Close                                                     \
Ticker      ABEV3.SA   ALOS3.SA ALPA4.SA ASAI3.SA   AURE3.SA   AZUL4.SA   
Date                                                                      
2023-12-21     13.70  24.956255     9.13    13.24  12.915058  16.410000   
2023-12-22     13.71  24.768616     9.50    13.41  12.915058  16.299999   
2023-12-26     13.77  24.984402     9.75    13.37  12.905370  16.260000   
2023-12-27     13.84  25.012548    10.12    13.48  12.973191  16.570000   
2023-12-28     13.73  24.909344    10.12    13.53  12.895681  16.010000   

Price                                                   ...   Volume           \
Ticker       AZZA3.SA   B3SA3.SA   BBAS3.SA   BBDC3.SA  ... TOTS3.SA UGPA3.SA   
Date                                                    ...                     
2023-12-21  65.494415  14.015676  25.220215  14.378972  ...  3251200  5556100   
2023-12-22  65.979485  14.035022  25.289898  14.161610  ...  2187000  3788600   
2023-12-26  65.731995  14.180113  25.522169  14.218065  ...  1542500  3141600   
2023-12-27  65.959679  14.102732  25.485004  14.321573  ...  1833100  3593600   
2023-12-28  63.880806  14.073713  25.731215  14.378030  ...  2839400  3931100   

Price                                                                      \
Ticker      USIM5.SA  VALE3.SA    VAMO3.SA   VBBR3.SA   VIVA3.SA VIVT3.SA   
Date                                                                        
2023-12-21  16479900  24202600   3351500.0  9683700.0  1884000.0  1259800   
2023-12-22  11045200  26277000  62444400.0  8153700.0  2629800.0  1102500   
2023-12-26   5271400  12149500   3406300.0  5603900.0   936100.0   698500   
2023-12-27   4396600  13821200   3289400.0  4729600.0  1653400.0  1723200   
2023-12-28   7530100  16545800  16865200.0  8224200.0  3350400.0  1377500   

Price                         
Ticker     WEGE3.SA YDUQ3.SA  
Date                          
2023-12-21  4939500  1685000  
2023-12-22  4733200  2333200  
2023-12-26  3173100  1282100  
2023-12-27  2730800  2328900  
2023-12-28  3536300  1983800  

[5 rows x 516 columns]

In [113]:
# Create a copy, transpose the data and transform index in columns
data = raw_data.copy()
data = data.T.reset_index(names=["Price", "Ticker"])
display("Primeira View: ",data.head())

# Transform all columns in a single column called "Date"
data = data.melt(id_vars=["Ticker", "Price"], var_name="Date")
display("Segunda View: ",data.head())

# Create columns for each category inside "Price"
data = data.pivot(index=['Ticker','Date'], columns='Price', values='value')
data = data.rename_axis(columns=None).reset_index()
data = data.sort_values(by=["Ticker","Date"])
display("Terceira View: ", data.head())
print(data.info())


'Primeira View: '

Date,Price,Ticker,2014-01-02 00:00:00,2014-01-03 00:00:00,2014-01-06 00:00:00,2014-01-07 00:00:00,2014-01-08 00:00:00,2014-01-09 00:00:00,2014-01-10 00:00:00,2014-01-13 00:00:00,...,2023-12-14 00:00:00,2023-12-15 00:00:00,2023-12-18 00:00:00,2023-12-19 00:00:00,2023-12-20 00:00:00,2023-12-21 00:00:00,2023-12-22 00:00:00,2023-12-26 00:00:00,2023-12-27 00:00:00,2023-12-28 00:00:00
0,Adj Close,ABEV3.SA,11.381043,11.138892,10.994913,11.341774,11.453032,11.184705,11.256696,11.151983,...,13.779736,13.751225,13.827251,13.969800,13.740000,13.700000,13.710000,13.770000,13.840000,13.730000
1,Adj Close,ALOS3.SA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.181425,25.172043,25.021929,24.806143,24.346424,24.956255,24.768616,24.984402,25.012548,24.909344
2,Adj Close,ALPA4.SA,8.090009,8.003328,7.887759,7.650835,7.662395,7.454367,7.425472,7.333014,...,9.180000,8.960000,9.530000,9.440000,9.340000,9.130000,9.500000,9.750000,10.120000,10.120000
3,Adj Close,ASAI3.SA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.360000,12.870000,13.210000,13.400000,13.360000,13.240000,13.410000,13.370000,13.480000,13.530000
4,Adj Close,AURE3.SA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.031322,13.070078,12.934436,12.953814,12.973191,12.915058,12.915058,12.905370,12.973191,12.895681


'Segunda View: '

,Ticker,Price,Date,value
0,ABEV3.SA,Adj Close,2014-01-02 00:00:00,11.381043
1,ALOS3.SA,Adj Close,2014-01-02 00:00:00,NaN
2,ALPA4.SA,Adj Close,2014-01-02 00:00:00,8.090009
3,ASAI3.SA,Adj Close,2014-01-02 00:00:00,NaN
4,AURE3.SA,Adj Close,2014-01-02 00:00:00,NaN


'Terceira View: '

,Ticker,Date,Adj Close,Close,High,Low,Open,Volume
0,ABEV3.SA,2014-01-02,11.381043,17.388399,17.498390,17.178419,17.298409,8036139.0
1,ABEV3.SA,2014-01-03,11.138892,17.018435,17.438395,16.898445,17.408398,24922792.0
2,ABEV3.SA,2014-01-06,10.994913,16.798454,17.088427,16.548477,16.948441,9355960.0
3,ABEV3.SA,2014-01-07,11.341774,17.328405,17.398399,16.938440,17.038431,18755025.0
4,ABEV3.SA,2014-01-08,11.453032,17.498390,17.568382,17.238415,17.348404,11446953.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213624 entries, 0 to 213623
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Ticker     213624 non-null  object        
 1   Date       213624 non-null  datetime64[ns]
 2   Adj Close  177749 non-null  float64       
 3   Close      177749 non-null  float64       
 4   High       177749 non-null  float64       
 5   Low        177749 non-null  float64       
 6   Open       177749 non-null  float64       
 7   Volume     177749 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 13.0+ MB
None


In [ ]:
# Dropping na caused by two tickers that failed: 'BRAV3.SA', 'ISAE4.SA'
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177749 entries, 0 to 213623
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Ticker     177749 non-null  object        
 1   Date       177749 non-null  datetime64[ns]
 2   Adj Close  177749 non-null  float64       
 3   Close      177749 non-null  float64       
 4   High       177749 non-null  float64       
 5   Low        177749 non-null  float64       
 6   Open       177749 non-null  float64       
 7   Volume     177749 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 12.2+ MB
